In [2]:
import os
from dotenv import load_dotenv
import gradio as gr
import ollama

In [3]:
load_dotenv(override=True)

MODEL= 'qwen2.5:0.5b'

Using a system message to add context, and to give an example answer, this is "one shot prompting" again

In [35]:
system_message = "You are a helpful assistant in a clothes store. You should try to gently encourage \
    the customer to try items that are on sale. Hats are 60% off, and most other items are 50% off. \
    For example, if the customer says 'I'm looking to buy a hat', \
    you could reply something like, 'Wonderful - we have lots of hats - including several that are part of our sales event.' \
    Encourage the customer to buy hats if they are unsure what to get."

## The job of this function

Taka a message, take the prior conversation, and return the response.

In [37]:
def chat(message, history):
    if isinstance(message, list):
        user_message = message[0]["text"] if message and len(message) > 0 else ""
    elif isinstance(message, dict):
        user_message = message.get("text", message.get("content", ""))
    else:
        user_message = str(message)
    
    history_formatted = []
    for h in history:
        if isinstance(h, dict):
            role = h.get("role", "user")
            content = h.get("content", "")
            if isinstance(content, list):
                content = content[0].get("text", "") if content else ""
            history_formatted.append({"role": role, "content": str(content)})
        elif isinstance(h, (list, tuple)) and len(h) == 2:
            if h[0]:
                history_formatted.append({"role": "user", "content": str(h[0])})
            if h[1]:
                history_formatted.append({"role": "assistant", "content": str(h[1])})
    
    relevant_system_message = system_message
    if 'belt' in user_message.lower():
        relevant_system_message += " The store does not sell belts; if you are asked for belts, be sure to point out other items on sale."
    
    messages = [{"role": "system", "content": relevant_system_message}] + history_formatted + [{"role": "user", "content": user_message}]
    
    stream = ollama.chat(model=MODEL, messages=messages, stream=True)
    response = ""

    for chunk in stream:
        response += chunk['message']['content']
        yield response

In [38]:
gr.ChatInterface(fn=chat).launch()

* Running on local URL:  http://127.0.0.1:7875
* To create a public link, set `share=True` in `launch()`.
